In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import re
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [2]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ejfel\AppData\Roaming\nltk_data...


In [3]:
with open('coffee_words.pickle','rb') as read_file:
    coffee = pickle.load(read_file)
with open('coffee_ratings.pickle','rb') as read_file:
    ratings = pickle.load(read_file)
with open('combined.pickle','rb') as read_file:
    combined = pickle.load(read_file)
with open('df.pickle','rb') as read_file:
    df = pickle.load(read_file)

In [4]:
sid = SentimentIntensityAnalyzer()

In [20]:
# df['blind'] = coffee.Review
df['scores'] = df.Text.apply(lambda Text: sid.polarity_scores(Text))
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

NameError: name 'blind' is not defined

In [8]:
df.sample(20)

,Roaster,Text,Two Topic,Three Topic,Four Topic,Five Topic,Six Topic,Seven Topic,Eight Topic,Ten Topic,Nine Topic,rating,scores,compound,comp_score
2151,Yes Bean Fresh Roast Café,Reviewed as espresso. Opulent and pungent; com...,1,1,1,1,1,1,1,1,1,93,"{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'comp...",0.6808,pos
592,Small Eyes Cafe,"Sweet, heavy, musky and perfumy. Elderflowers,...",0,0,3,3,3,3,3,5,5,92,"{'neg': 0.024, 'neu': 0.808, 'pos': 0.168, 'co...",0.9667,pos
4209,Atomic Cafe Coffee Roasters,"A refreshing, meadowy coffee. Sweet flowering ...",1,2,2,2,5,5,5,2,2,93,"{'neg': 0.0, 'neu': 0.789, 'pos': 0.211, 'comp...",0.9716,pos
1310,Espresso Republic,"Deeply rich, bittersweet, herbaceous. Almond b...",0,0,3,4,4,3,3,5,5,93,"{'neg': 0.024, 'neu': 0.903, 'pos': 0.074, 'co...",0.8288,pos
5735,Green Mountain Coffee Roasters,The center of the profile is satisfyingly roun...,1,1,1,1,1,1,7,7,7,79,"{'neg': 0.036, 'neu': 0.825, 'pos': 0.139, 'co...",0.9693,pos
2311,PT's Coffee Roasting Co.,"Delicate but exotic, juicy, brandy-toned. Blue...",0,0,0,0,0,0,4,4,4,94,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...",0.9674,pos
2766,Home Barista Workshop,"Complexly fruit- and cider-toned. Shifting, la...",0,0,3,4,4,4,4,4,4,93,"{'neg': 0.0, 'neu': 0.935, 'pos': 0.065, 'comp...",0.7964,pos
1361,Kustomcoffee,"Roast-toned; musky and earthy. Raisin, moist p...",1,1,3,3,3,3,7,7,7,90,"{'neg': 0.0, 'neu': 0.925, 'pos': 0.075, 'comp...",0.8641,pos
1987,Wonderstate Coffee,"Sweetly and gently pungent, cocoa-toned. Roast...",0,0,3,3,3,3,3,5,5,92,"{'neg': 0.014, 'neu': 0.84, 'pos': 0.146, 'com...",0.9217,pos
2078,Victrola Coffee Roasters,"Rich, deep, yet delicate. Tangerine, rose-like...",1,1,1,3,3,6,6,7,6,93,"{'neg': 0.0, 'neu': 0.715, 'pos': 0.285, 'comp...",0.9743,pos


In [9]:
df.comp_score.value_counts()

pos    5850
neg     109
Name: comp_score, dtype: int64

In [17]:
df[df.comp_score == 'neg']

,Roaster,Text,Two Topic,Three Topic,Four Topic,Five Topic,Six Topic,Seven Topic,Eight Topic,Ten Topic,Nine Topic,rating,scores,compound,comp_score
406,Cafe Grumpy,"Richly sweet-tart, earth-toned. Blood orange, ...",0,0,3,3,3,3,3,5,5,93,"{'neg': 0.06, 'neu': 0.902, 'pos': 0.038, 'com...",-0.5267,neg
495,Cameron’s Coffee,"Balanced, sweet-toned. Baking chocolate, almon...",1,1,1,1,1,1,1,1,1,89,"{'neg': 0.017, 'neu': 0.983, 'pos': 0.0, 'comp...",-0.0772,neg
978,Maxim Coffee,Evaluated at proportions of 5 grams of instant...,1,1,1,1,1,1,1,1,1,72,"{'neg': 0.057, 'neu': 0.901, 'pos': 0.042, 'co...",-0.2732,neg
980,Yuban,Evaluated at proportions of 5 grams of instant...,1,1,3,1,1,3,3,5,5,68,"{'neg': 0.08, 'neu': 0.877, 'pos': 0.043, 'com...",-0.3923,neg
1262,A.R.C.,"Evaluated as espresso. Fir, almond, baker’s ch...",1,1,1,1,1,1,1,1,1,88,"{'neg': 0.054, 'neu': 0.919, 'pos': 0.027, 'co...",-0.0516,neg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5910,Baja Vista,A powerful but flawed coffee. Coffee professio...,1,2,0,0,2,2,7,7,7,77,"{'neg': 0.117, 'neu': 0.81, 'pos': 0.073, 'com...",-0.8126,neg
5920,"Safeway, Inc.","For a moment, somewhere between nose and finis...",1,1,1,1,1,1,7,7,7,70,"{'neg': 0.167, 'neu': 0.747, 'pos': 0.086, 'co...",-0.8832,neg
5921,Lucky Stores,The vanilla / nut tones in the aroma fade quic...,1,1,1,1,1,1,7,7,7,68,"{'neg': 0.124, 'neu': 0.783, 'pos': 0.094, 'co...",-0.4404,neg
5923,Eight O'Clock Coffee,"Fermented notes, barely detectable in the acid...",1,1,1,1,2,2,7,7,7,60,"{'neg': 0.085, 'neu': 0.892, 'pos': 0.023, 'co...",-0.5494,neg


In [19]:
df.iloc[406].Text

'Richly sweet-tart, earth-toned. Blood orange, pipe tobacco, roasted almond, rhododendron, baking chocolate in aroma and cup. Tart-leaning structure with crisp, green-apple-like (malic) acidity; deep, velvety mouthfeel. The richly drying finish consolidates to blood orange and pipe  coffee tied for the fourth-highest rating in a cupping of “desert-island coffees” submitted by roasters for Coffee Review’s May 2020 tasting report. Muungano, the name of this cooperative, means “togetherness” in Swahili. One-third of the coop’s members are women, many of whom were widowed by war. Produced entirely from trees of strains of the Bourbon variety of Arabica and processed by the washed method (fruit skin and pulp are removed before drying). Certified organic. An integral part of New York’s coffee renaissance, Brooklyn-based Café Grumpy opened its first café in 2005 and has since expanded to 10 locations in New York, including a roasting facility, and one in Miami. '